In [44]:
import polars as pl
import altair as alt

personal_info = pl.read_csv("data/6_personal_info.csv")

In [45]:
from attributes import *
#
# 표
# 

keys = {
    "성별" : {
        1 : "남성",
        2 : "여성"
    }, 
    "연령대": {
        1 : "20대",
        2 : "30대",
        3 : "40대",
        4 : "50대",
        5 : "60대 이상",
    },
    "직위" : {
        1 : "책임급",
        2 : "선임급",
        3 : "원급", 
        4 : "무기(선임급)",
        5 : "무기(원급)",
        6.: "기타 직위"
    },
    "최종학위" : {
        1 : "학사",
        2 : "석사",
        3 : "박사",
    },
    "연구경력": {
        1 : "최종 학위 취득 후 5년 미만",
        2 : "5년 이상 10년 미만",
        3 : "10년 이상 15년 미만",
        4 : "15년 이상",
    },
    "연구분야": {
        1 : "해양물리",
        2 : "해양화학",
        3 : "해양생물",
        4 : "해양지질",
        5 : "해양공학",
        6 : "해양정책",
        7 : "기타분야",
    },
}

df = pl.DataFrame()
for key, value in MAP_PERSON.items():
    new_df = personal_info.group_by(value).len().sort(value).rename({value:"info"})
    new_df = new_df.with_columns(
        pl.col("info").map_elements(lambda x : f"{keys[key][x]}", return_dtype=pl.String),
        pl.col("info").map_elements(lambda _ : f"{key}", return_dtype=pl.String).alias("category")
    )
    df = df.vstack(new_df)

df.write_csv("figure/1/조사대상의특성.csv")

df.group_by("category").agg(
    (pl.col("len") / pl.col("len").sum())
).write_json("figure/1/조사대상의특성.json")

In [46]:
bar = df.plot.bar().encode(
    x=alt.X("len:Q", title=None),
    y=alt.Y("info:N", title=None).sort(keys.keys()),
    color=alt.Color("category", title=None).sort(keys.keys())
).properties(
    width=500,
    height=compute_height(500),
    title="조사 대상의 특성"
)

text = bar.mark_text(
    align='left',  
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='len:Q',
    color=alt.value("black"),
    detail="info:N"
)

bar = bar + text
bar.save("figure/1/조사대상의특성.png")